## Names:

- Mohammed Afif Chifaoui

- Ricardo Vazquez Alvarez


# Homework 2 Discrete and Non Linear Optimization.


In [3]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
import math
import pandas


We want to find the max flow between countries in order to export the amount of olive oil that each country requires.

For this we can create the following variables:

- C = Set of Countries (destination countries and the in between countries)
    - (**Destinations:** Spain, Ireland, Poland, Italy, Switzerland ; **In between:** France, Monaco, UK, Belgium, Netherlands, and Germany).

- P = Set of destination pairs (The connecting countries for flow)
    

- M - The set of minimum amount of olive oil required for the destination countries.




In [129]:
%%writefile olive.dat
set C := Spain Ireland Poland Italy Switzerland France Monaco UK Belgium Netherlands Germany;
set P := (Spain, France) (France, Monaco) (France, Italy) (France, Switzerland) (France, Belgium) (France, UK) (Belgium, Netherlands) (Belgium, Germany) (France, Germany) (UK, Ireland) (Netherlands, Germany) (Germany, Poland);

param origin := Spain;


param maximum :=
Spain France 200
France Monaco 20
France Italy 150
France Switzerland 40
France Belgium 40
France UK 50
Belgium Netherlands 30
Belgium Germany 40
France Germany 30
UK Ireland 50
Netherlands Germany 20
Germany Poland 60;

param needs :=
Ireland 1
Poland 1
Italy 1
Switzerland 1;



Overwriting olive.dat


In [130]:
%%writefile olive_exp.py
from pyomo.environ import * 

model = AbstractModel()

model.C = Set()

# Initialize the Points with the cartesian product of the countries we have.
model.P = Set(within = model.C*model.C)

# Spain is the starting point
model.origin = Param(within = model.C)


model.maximum = Param(model.P)

model.needs = Param(model.C)

model.flow = Var(model.P, within = NonNegativeReals)


# Objective Function

def objective_rule(model):

    obj_val = sum(model.flow[origin,dest] for origin,dest in model.P if dest =='Ireland' or dest =='Poland' or dest == 'Italy' or dest =='Switzerland')

    return obj_val

model.obj = Objective(rule = objective_rule, sense = maximize)


def allowed_quantity(model, m, n):
    return model.flow[m, n] <= model.maximum[m, n]

def flow_rule(model , k):
    # if it is a final destination - dont flow out . If it is a 
    if k =='Ireland' or k =='Poland' or k == 'Italy' or k =='Switzerland' or k ==value(model.origin):
        return Constraint.Skip
    
    # the inflow from one origin to a destination MUST equal the outflow from origin to destination.
    # calculating the inflow
    inflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if dest==k)
    # calculating the outflow.
    outflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if origin==k)

    return inflow==outflow

def requested_oil(model, i):
    if i== "Italy" or i== "Switzerland" or i== "Ireland" or i== "Poland":
        return sum(model.flow[j, k] for (j,k) in model.P if k==i)>=model.needs[i]
    else:
        return Constraint.Skip


model.flow_rule = Constraint(model.C, rule = flow_rule)
model.allowed_quantity = Constraint(model.P, rule = allowed_quantity)
model.requested_oil = Constraint(model.C, rule = requested_oil)
    

Overwriting olive_exp.py


In [128]:
instance = model.create_instance("olive.dat")

# Solve the model
# destination
solver = SolverFactory("glpk")
solver.solve(instance, tee=True)

# Print the results
instance.display()

GLPSOL: GLPK LP/MIP Solver, v4.58
Parameter(s) specified in the command line:
 --write C:\Users\Ricardo\AppData\Local\Temp\tmpluw35sx2.glpk.raw --wglp C:\Users\Ricardo\AppData\Local\Temp\tmpas6imrkz.glpk.glp
 --cpxlp C:\Users\Ricardo\AppData\Local\Temp\tmpramjxtve.pyomo.lp
Reading problem data from 'C:\Users\Ricardo\AppData\Local\Temp\tmpramjxtve.pyomo.lp'...
22 rows, 12 columns, 35 non-zeros
126 lines were read
Writing problem data to 'C:\Users\Ricardo\AppData\Local\Temp\tmpas6imrkz.glpk.glp'...
92 lines were written
GLPK Simplex Optimizer, v4.58
22 rows, 12 columns, 35 non-zeros
Preprocessing...
4 rows, 8 columns, 13 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 4
      0: obj =  -0.000000000e+00 inf =   4.000e+00 (2)
      2: obj =   4.000000000e+00 inf =   0.000e+00 (0)
*     5: obj =   2.000000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOU

Model unknown

  Variables:
    flow : Size=12, Index=P
        Key                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Belgium', 'Germany') :     0 :   9.0 :  None : False : False : NonNegativeReals
        ('Belgium', 'Netherlands') :     0 :   0.0 :  None : False : False : NonNegativeReals
             ('France', 'Belgium') :     0 :   9.0 :  None : False : False : NonNegativeReals
             ('France', 'Germany') :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('France', 'Italy') :     0 : 150.0 :  None : False : False : NonNegativeReals
              ('France', 'Monaco') :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('France', 'Switzerland') :     0 :  40.0 :  None : False : False : NonNegativeReals
                  ('France', 'UK') :     0 :   1.0 :  None : False : False : NonNegativeReals
             ('Germany', 'Poland') :     0 :   9.0 :  None : False : False : NonNegativeReals
        ('Neth

In [131]:
!pyomo solve  olive_exp.py  olive.dat --solver=glpk --summary 

!type results.yml

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.02] Applying solver
[    0.15] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 200.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    flow : Size=12, Index=P
        Key                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Belgium', 'Germany') :     0 :   9.0 :  None : False : False : NonNegativeReals
        ('Belgium', 'Netherlands') :     0 :   0.0 :  None : False : False : NonNegativeReals
             ('France', 'Belgium') :     0 :   9.0 :  None : False : False : NonNegativeReals
             ('France', 'Germany') :     0 :   0.0 :  None : False : False : NonNegativeReals
               ('France', 'Italy') :     0 : 150.0 :  None : False : False : NonNegativeReals
              ('France', 'Monaco') :     0 : 

# Work

In [26]:
# Objective Function
def objective_rule(model):

    obj_val = sum(model.flow[origin,dest] for origin,dest in model.T if dest =='Ireland' or dest =='Poland' or dest == 'Italy' or dest =='Switzerland')

    return obj_val

model.obj = Objective(rule = objective_rule, sense = maximize)
    

Overwriting flow.py


## Constraints

In [ ]:
 # Constraint 1: do not exceed the maximum quantity of olive allowed in a connection

def allowed_quantity(model, m, n):
 return model.flow[m, n] <= model.maximum[m, n]

In [ ]:
def flow_rule(model , k):
    # if it is a final destination - dont flow out . If it is a 
    if k =='Ireland' or k =='Poland' or k == 'Italy' or k =='Switzerland' or k ==value(model.origin):
        return Constraint.Skip
    
    # the inflow from one origin to a destination MUST equal the outflow from origin to destination.
    # calculating the inflow
    inflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if dest==k)
    # calculating the outflow.
    outflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if origin==k)

    return inflow==outflow
        

In [ ]:
# Constraint 3: ensure that each final destination is receiving requested amount of oil

def requested_oil(model, i):
    if i== "Italy" or i== "Switzerland" or i== "Ireland" or i== "Poland":
        return sum(model.flow[j, k] for (j,k) in model.P if k==i)>=model.needs[i]
    else:
        return Constraint.Skip

In [ ]:
model.flow_rule = Constraint(model.C, rule = flow_rule)
model.allowed_quantity = Constraint(model.P, rule = allowed_quantity)
model.requesedt_oil = Constraint(model.C, rule = requested_oil)


# Bibliography




https://oec.world/es/profile/bilateral-product/olive-oil-virgin/reporter/pol